# CI Portfolio Project 5 - Filter Maintenance Predictor 2022
## **ML Model - Predict Remaining Useful Life (RUL)**

## Objectives

Answer [Business Requirement 1](https://github.com/roeszler/filter-maintenance-predictor/blob/main/README.md#business-requirements) :
*   Fit and evaluate a **regression model** to predict the Remaining Useful Life of a replaceable part
*   Fit and evaluate a **classification model** to predict the Remaining Useful Life of a replaceable part should the regressor not perform well.

## Inputs

Data cleaning:
* outputs/datasets/cleaned/dfCleanTotal.csv

## Outputs

* Train set (features and target)
* Test set (features and target)
* Validation set (features and target)
* ML pipeline to predict RUL
* A map of the labels
* Feature Importance Plot



---

### Change working directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("Current directory set to new location")

In [ ]:
current_dir = os.getcwd()
current_dir

---

## The major steps in this Regressor Pipeline

<details>
<summary style="font-size: 0.9rem;"><strong>1. ML Pipeline: Regressor</strong> (Dropdown List)</summary>

* Create Regressor Pipeline
* Split the train set
* Grid Search CV SKLearn
    * Use standard hyperparameters to find most suitable algorithm
    * Extensive search on most suitable algorithm to find the best hyperparameter configuration
* Assess Feature Performance
* Evaluate Regressor
* Create Train, Test, Validation Sets
</details></br>


<details>
<summary style="font-size: 0.9rem;"><strong>2. ML Pipeline: Regressor + Principal Component Analysis</strong> (PCA)</summary>

* Prepare the Data for the Pipeline
* Create Regressor + PCA Pipeline
* Split the train and validation sets
* Grid Search CV SKLearn
    * Use standard hyperparameters to find most suitable algorithm
    * Do an extensive search on most suitable algorithm to find the best hyperparameter configuration
* Assess Feature Performance
* Evaluate Regressor
* Create Train, Test, Validation Sets
</details></br>

<details>
<summary style="font-size: 0.9rem;"><strong>3. Convert Regression to Classification</strong> (Optionally)</summary>

* Convert numerical target to bins, and check if it is balanced
* Rewrite Pipeline for ML Modelling
* Load Algorithms For Classification
* Split the Train Test sets:
* Grid Search CV SKLearn:
    * Use standard hyper parameters to find most suitable model
    * Grid Search CV
    * Check Result
* Do an extensive search on the most suitable model to find the best hyperparameter configuration.
    * Define Model Parameters
    * Extensive Grid Search CV                             
    * Check Results
    * Check Best Model
    * Parameters for best model
    * Define the best clf_pipeline
* Assess Feature Importance
* Evaluate Classifier on Train and Test Sets
    * Custom Function
    * List that relates the classes and tenure interval
</details></br>

<details><summary style="font-size: 0.9rem;"><strong>4. Decide which pipeline to use</strong></summary></details></br>

<details>
<summary style="font-size: 0.9rem;"><strong>5. Refit with the best features</strong></summary>

* Rewrite Pipeline
* Split Train Test Set with only best features
* Subset best features
* Grid Search CV SKLearn
* Best Parameters
    * Manually
* Grid Search CV
* Check Results
* Check Best Model
* Define the best pipeline
</details></br>

<details><summary style="font-size: 0.9rem;"><strong>6. Assess Feature Importance</strong></summary></details></br>

<details><summary style="font-size: 0.9rem;"><strong>7. Push Files to Repo</strong></summary></details>

<!-- Modelling:
The hypothesis part of the process where you will find out whether you can answer the question.
* Identify what techniques to use.
* Split your data into train, validate and test sets.
* Build and train the models with the train data set.
* Validate Models and hyper-parameter : Trial different machine learning methods and models with the validation data set.
* Poor Results - return to data preparation for feature engineering
* Successful hypothesis - where the inputs from the data set are mapped to the output target / label appropriately to evaluate.

5. Evaluation:
Where you test whether the model can predict unseen data.
* Test Dataset
* Choose the model that meets the business success criteria best.
* Review and document the work that you have done.
* If your project meets the success metrics you defined with your customer?
- Ready to deploy. -->

---

### Load Cleaned Data
Target variable for regressor, remove from classifier and drop other variables not required

In [ ]:
import numpy as np
import pandas as pd

df_total = pd.read_csv(f'outputs/datasets/transformed/dfTransformedTotal.csv') # data with all negative log_EWM values removed
df_total_model = (pd.read_csv('outputs/datasets/transformed/dfTransformedTotal.csv')
        .drop(labels=['4point_EWM', 'change_DP', 'change_EWM'], axis=1)
    )
df_train_even_dist = (pd.read_csv(f'outputs/datasets/transformed/dfTransformedTrain.csv')
        .drop(labels=['4point_EWM', 'change_DP', 'change_EWM', 'std_DP', 'median_DP', 'bin_size'], axis=1)
    )
print(df_total.shape, '= df_total')
print(df_total_model.shape, '= df_total_model')
print(df_train_even_dist.shape, '= df_train_even_dist')
df_total

In [ ]:
df_total_model

---

# ML Pipeline : Regressor
## Create Regressor Pipeline
### Set the Transformations
* Smart correlation
* feat_scaling
* feat_selection
* Modelling
* Model as variable

Note: Numerical Transformation not required as data supplied as integers

In [ ]:
df_total_model

## Split the data into Train, Test, Validate

Data is discrete however in bins, so:
#### Define Cleaned **Train** & **Test** Datasets

In [ ]:
df_total_model

In [ ]:
n = df_total_model['Data_No'].iloc[0:len(df_total)]
# df_train = df_total_model[n < 51].reset_index(drop=True)
df_test = df_total_model[n > 50].reset_index(drop=True)
df_train = df_train_even_dist
df_train_model = df_train_even_dist
df_train

In [ ]:
df_test

#### Determine **Target** and **Independent** Variables and Extract **Validation** Dataset

As discussed in the readme, this data has been supplied pre-split into **train** and **test** within unique **data bins**. 
We extract random observations from the **test** dataset to create a **validation** set, in a 70:30 split.


In [ ]:
df_test

Review correlations, Drop Features and Split into **70% test** and **30% validate**. 

In [ ]:
from sklearn.model_selection import train_test_split
from feature_engine.selection import SmartCorrelatedSelection

corr_sel = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")
df_engineering = df_test.copy()
corr_sel.fit_transform(df_engineering)

features_to_drop_test = corr_sel.features_to_drop_
X = df_test.drop(features_to_drop_test,axis=1)
y = df_test['Differential_pressure']

X_test, X_validate, y_test, y_validate = train_test_split(X,y,test_size=0.30, random_state=0)

print(X_test.shape, 'X_test')
print(X_validate.shape, 'X_validate')
print(y_test.shape, 'y_test')
print(y_validate.shape, 'y_validate')
print('\nFeatures Dropped :\n', corr_sel.features_to_drop_)

In [ ]:
X_test

#### Define **X_train**, **y_train** variables

In [ ]:
df_train

Create **train** dataset with the same variables dropped as the **test** dataset

In [ ]:
# corr_sel = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")
# df_engineering = df_train.copy()
# corr_sel.fit_transform(df_engineering)

X_train = df_train.drop(features_to_drop_test,axis=1)
y_train = df_train['Differential_pressure']

print(X_train.shape, 'X_train')
print(y_train.shape, 'y_train')
print('\nFeatures Dropped :\n', features_to_drop_test)

In [ ]:
X_train

In [ ]:
y_train

## Handling Target Imbalance
### No need to handle target imbalance in this **regression model**.
* Typically we only need to create a single pipeline for Classification or Regression task. 
* The exception occurs when we need to handle a **classification target imbalance**, which requires more than one model to process 

#### ML Pipeline for **Fitting Models** (regression)
Import features & models

In [ ]:
# Feature Management
from sklearn.pipeline import Pipeline
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.feature_selection import SelectFromModel

# ML regression algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

Define the pipeline

In [ ]:
def PipelineOptimization(model):
    pipeline_base = Pipeline([
        # ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
        #                                              variables=['Differential_pressure', 'Flow_rate',
        #                                                         # 'log_EWM', 'Time', 'mass_g', 'Tt', 'filter_balance',
        #                                                         'Dust_feed', 'Dust', 'cumulative_mass_g'])),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")),
        ("feat_scaling", StandardScaler()),
        ("feat_selection",  SelectFromModel(model)),
        ("model", model),
    ])
    return pipeline_base

#### **Custom Class** to fit a set of algorithms, each with its own set of hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key]) # the model

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score (R²)'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score (R²)': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score (R²)', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches


### Use standard hyperparameters to find most suitable algorithm for the data

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    'DecisionTreeRegressor': DecisionTreeRegressor(random_state=0),
    'RandomForestRegressor': RandomForestRegressor(random_state=0),
    'ExtraTreesRegressor': ExtraTreesRegressor(random_state=0),
    'AdaBoostRegressor': AdaBoostRegressor(random_state=0),
    'GradientBoostingRegressor': GradientBoostingRegressor(random_state=0),
    'XGBRegressor': XGBRegressor(random_state=0),
    'SGDRegressor': SGDRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    'DecisionTreeRegressor': {},
    'RandomForestRegressor': {},
    'ExtraTreesRegressor': {},
    'AdaBoostRegressor': {},
    'GradientBoostingRegressor': {},
    'XGBRegressor': {},
    'SGDRegressor': {},
}

#### Fit the pipelines, using the above models with **default hyperparameters** to find the most suitable model
* Parsed the train set
* Set the performance metric as an R2 score (Regression: described in our ML business case)
* Cross validation as 5 (rule of thumb)

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_quick, grid_search_pipelines_quick = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_quick

In [ ]:
X_train

#### Observations

* The average **R² score** (mean_score) ranges from **0.98 to 1**, which is exceptional. This indicates how well a model of the data fits the actual data.
* A value of R² score = 1 represents a perfect fit.
* This is much higher than the **0.7** tolerance we decided in the business case.
* The best result is the **LinearRegression**, **ExtraTreesRegressor**, and/or the **SGDRegressor** however all algorithms can be confidently used to train a model.
<!-- * We will perform an extensive search to hopefully improve performance. -->

---

### Do an extensive search on the most suitable model to find the best hyperparameter configuration.
Define model and parameters for each extensive search

#### Linear Regressor Model

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

models_search = {
    'LinearRegression': LinearRegression(),
}

params_search = {
    'LinearRegression':{
        'model__fit_intercept': [False, True],
        'model__positive': [False, True],
        'model__copy_X': [True],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_linear, grid_search_pipelines_linear = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_linear

#### SGDRegressor Model

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

models_search = {
    'SGDRegressor': SGDRegressor(),
}

params_search = {
    'SGDRegressor':{
        'model__loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
        'model__penalty': ['l2', 'l1', 'elasticnet', None],
        'model__alpha': [0.0001,0.0002,0.0003],
        'model__learning_rate': [1e-1,1e-2,1e-3, 'optimal', 'adaptive'],
        'model__fit_intercept': [False, True],
        'model__early_stopping': [True],
        'model__average': [True],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_SGDR, grid_search_pipelines_SGDR = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_SGDR

Concatenation to summary

In [ ]:
grid_search_pipelines_SGDR

In [ ]:
grid_search_summary = pd.concat([grid_search_summary_linear, grid_search_summary_SGDR], ignore_index=True)
grid_search_pipelines = dict(grid_search_pipelines_linear); grid_search_pipelines.update(grid_search_pipelines_SGDR)

#### ExtraTreesRegressor Model

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html

models_search = {
    'ExtraTreesRegressor': ExtraTreesRegressor(),
}

params_search = {
    'ExtraTreesRegressor':{
        'model__n_estimators': [100,200,300],
        'model__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        'model__max_depth': [None],
        'model__min_samples_split': [2,4,6],
        'model__max_features': [1.0, 'sqrt', 'log2'],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=2)

Check results

In [ ]:
grid_search_summary_ExtraTrees, grid_search_pipelines_ExtraTrees = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_ExtraTrees

Concatenation to summary

In [ ]:
grid_search_summary = pd.concat([grid_search_summary, grid_search_summary_ExtraTrees], ignore_index = True)
data = dict(grid_search_pipelines); data.update(grid_search_pipelines_ExtraTrees)
grid_search_pipelines = data

#### RandomForestRegressor Model

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

models_search = {
    'RandomForestRegressor': RandomForestRegressor(),
}

params_search = {
    'RandomForestRegressor':{
        'model__n_estimators': [100,200,300],
        'model__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        'model__max_depth': [None],
        'model__max_features': [1.0, 'sqrt', 'log2'],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=2)

Check results

In [ ]:
grid_search_summary_RForest, grid_search_pipelines_RForest = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_RForest

Concatenation to summary

In [ ]:
grid_search_summary = pd.concat([grid_search_summary, grid_search_summary_RForest], ignore_index = True)
data = dict(grid_search_pipelines); data.update(grid_search_pipelines_RForest)
grid_search_pipelines = data

#### DecisionTreeRegressor Model

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

models_search = {
    'DecisionTreeRegressor': DecisionTreeRegressor(),
}

params_search = {
    'DecisionTreeRegressor':{
        'model__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        'model__splitter': ['best', 'random'],
        'model__max_depth': [None],
        'model__min_samples_split': [2,4,6],
        'model__max_features': [1.0, 'sqrt', 'log2', 0.3, None],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_DTree, grid_search_pipelines_DTree = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_DTree

Concatenation to summary

In [ ]:
grid_search_summary = pd.concat([grid_search_summary, grid_search_summary_DTree], ignore_index=True)
data = dict(grid_search_pipelines); data.update(grid_search_pipelines_DTree)
grid_search_pipelines = data

In [ ]:
grid_search_pipelines.keys()

#### XGBRegressor Model

In [ ]:
# documentation to help on hyperparameter list: 
# https://xgboost.readthedocs.io/en/latest/parameter.html

models_search = {
    'XGBRegressor': XGBRegressor(),
}

params_search = {
    'XGBRegressor':{
        'model__max_depth': [2, 4, 6],
        'model__n_estimators': [50, 100, 200],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_XGBR, grid_search_pipelines_XGBR = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_XGBR

Concatenation to summary

In [ ]:
grid_search_summary = pd.concat([grid_search_summary, grid_search_summary_XGBR], ignore_index = True)
data = dict(grid_search_pipelines); data.update(grid_search_pipelines_XGBR)
grid_search_pipelines = data

#### GradientBoostingRegressor Model

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

models_search = {
    'GradientBoostingRegressor': GradientBoostingRegressor(),
}

params_search = {
    'GradientBoostingRegressor':{
        'model__loss': ['squared_error', 'huber', 'absolute_error', 'quantile'],
        'model__learning_rate': [1e-1,1e-2,1e-3],
        'model__n_estimators': [100, 200, 300],
        'model__criterion': ['friedman_mse', 'squared_error'],
        'model__max_features': [1.0, 'sqrt', 'log2'],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_GBR, grid_search_pipelines_GBR = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_GBR

Concatenation to summary

In [ ]:
grid_search_summary = pd.concat([grid_search_summary, grid_search_summary_GBR], ignore_index = True)
data = dict(grid_search_pipelines); data.update(grid_search_pipelines_GBR)
grid_search_pipelines = data

#### AdaBoostRegressor Model

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

models_search = {
    'AdaBoostRegressor': AdaBoostRegressor(),
}

params_search = {
    'AdaBoostRegressor':{
        'model__n_estimators': [50, 100, 200, 300],
        'model__learning_rate': [1, 1e-1,1e-2,1e-3],
        'model__loss': ['linear', 'square', 'exponential'],
        'model__random_state': [None],
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_ABR, grid_search_pipelines_ABR = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_ABR.head(10)

Concatenation to summary

In [ ]:
grid_search_summary = pd.concat([grid_search_summary, grid_search_summary_ABR], ignore_index = True)
data = dict(grid_search_pipelines); data.update(grid_search_pipelines_ABR)
grid_search_pipelines = data

Models Reviewed

In [ ]:
grid_search_summary_quick

## Observations
### Best model from quick search

In [ ]:
best_model = grid_search_summary_quick.iloc[0, 0]
best_model

Best model of each group

In [ ]:
grid_search_summary

In [ ]:
grid_search_summary = grid_search_summary.drop(labels=['model__copy_X', 'model__fit_intercept', 'model__positive',
                                                    'model__alpha', 'model__average', 'model__learning_rate', 'model__loss',
                                                    'model__penalty', 'model__criterion', 'model__max_depth', 'model__max_features',
                                                    'model__min_samples_split', 'model__splitter', 'model__n_estimators',
                                                    'model__random_state', 'model__early_stopping'], axis=1)
                                                    
grid_search_summary[grid_search_summary.estimator != grid_search_summary.estimator.shift(1)].head()

Parameters of each pipeline

In [ ]:
grid_search_pipelines.keys()

### Best Model Parameters

In [ ]:
# best_parameters = grid_search_pipelines_linear[best_model].best_params_
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

In [ ]:
X_train

In [ ]:
y_train

### Best Regressor based on search

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
# best_regressor_pipeline.keys()
best_regressor_pipeline

## Assess Feature Performance
To find the most important features in this pipeline. Since the best model is linear regression, we cannot access these features using **.features_importances**.
* The "best features" information is found in the pipeline's "feature selection" step as a boolean list.
* We can use this list to subset the train set columns.
* We create a DataFrame that contains these features' importance and plot it as a bar plot.

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# import numpy as np
# %matplotlib inline

# sns.set_style('whitegrid')

# # after data cleaning and feature engineering, the features may have changes
# # how many data cleaning and feature engineering steps does your pipeline have?
# data_cleaning_feat_eng_steps = 2
# columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
#                                         .transform(X_train)
#                                         .columns)

# best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support(
# )].to_list()

# # create DataFrame to display feature importance
# df_feature_importance = (pd.DataFrame(data={
#     'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],
#     'Importance': best_regressor_pipeline['model'].feature_importances_})
#     .sort_values(by='Importance', ascending=False)
# )

# # Most important features statement and plot
# print(f"* These are the {len(best_features)} most important features in descending order. "
#       f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

# df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
# plt.show()


In [ ]:
# # since we don't have feature selection step in this pipeline, best_features is Xtrain columns
# best_features = X_train.columns.to_list()
# # create a DataFrame to display feature importance
# df_feature_importance = (pd.DataFrame(data={
#    'Feature': best_features,
#    'Importance': best_regressor_pipeline['model'].feature_importances_})
#    .sort_values(by='Importance', ascending=False)
# )
# best_features = df_feature_importance['Feature'].to_list()
# # Most important features statement and plot
# print(f"* These are the {len(best_features)} most important features in descending order. "
#      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")
# df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
# plt.show()

## Evaluate Regressor, Train and Test Set Performance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)


def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:', np.sqrt(
        mean_squared_error(y, prediction)).round(3))
    print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    plt.show()

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

## Train the Model

Multiple regression and classification models under consideration 

* sklearn.linear_model.**LinearRegression**(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
* sklearn.linear_model.**LogisticRegression**(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
    * *.predict_proba(X)*
* sklearn.linear_model.**SGDRegressor**(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
    * *.SGDClassifier()*

List full of available and under consideration can be seen at scikitlearn [linear models](https://scikit-learn.org/stable/modules/linear_model.html#)

* No one optimal model. the most appropriate seems .LogisticRegression()
<!-- 
**.LinearRegression()** - Ordinary Least Squares
**.SGDClassifier()** and **.SGDRegressor()** - Stochastic Gradient Descent - SGD
.Ridge() 
.Lasso()
.MultiTaskLasso()
.ElasticNet()
.MultiTaskElasticNet()
.Lars() - Least Angle Regression
.LassoLars()
.OrthogonalMatchingPursuit() and orthogonal_mp()
.BayesianRidge() - Bayesian Regression
.ARDRegression() - Automatic Relevance Determination
Generalized Linear Models
**.LogisticRegression()** + **.predict_proba(X)**
.TweedieRegressor()
.Perceptron()
.PassiveAggressiveClassifier() and .PassiveAggressiveRegressor()
Robustness regression: outliers and modeling errors
.RANSACRegressor()
.TheilSenRegressor() and 
.HuberRegressor()
.QuantileRegressor()
Polynomial regression: extending linear models with basis functions
.PolynomialFeatures() transformer -->


Models with **R² score** = 1.0

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,y_train)

print('Intercept :', linreg.intercept_)
print('Coefficients :\n', linreg.coef_)
linreg

In [ ]:
from sklearn.linear_model import SGDRegressor
SGDreg = SGDRegressor()
SGDreg.fit(X_train,y_train)
print('Intercept :', SGDreg.intercept_)
print('Coefficients :\n', SGDreg.coef_)
SGDreg

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
XTRreg = ExtraTreesRegressor()
XTRreg.fit(X_train,y_train)

Subsequent Models

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFreg = RandomForestRegressor()
RFreg.fit(X_train,y_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DTreg = DecisionTreeRegressor()
DTreg.fit(X_train,y_train)

In [ ]:
from xgboost import XGBRegressor
XGBreg = XGBRegressor()
XGBreg.fit(X_train,y_train)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GBreg = GradientBoostingRegressor()
GBreg.fit(X_train,y_train)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ABreg = AdaBoostRegressor()
ABreg.fit(X_train,y_train)

### Predictions and Model Evaluation
A good metric for this is the **coefficient of determination** also called **r2-score**.
* Consider Cross validation

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

from sklearn.metrics import r2_score
prediction_linear = linreg.predict(X_test)
prediction_SGDreg = SGDreg.predict(X_test)

print('prediction_linreg :', r2_score(y_test,prediction_linear))
print('prediction_SGDreg :', round(r2_score(y_test,prediction_SGDreg)))
print('prediction_SGDreg :', r2_score(y_test,prediction_SGDreg))


In [ ]:
# from sklearn.metrics import classification_report

# prediction = linreg.predict(X_test)
# print(classification_report(y_test,prediction))

---

## Save Datasets 
Save the files to **/models** folder

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/models/RULRegressor')
except Exception as e:
  print(e)

grid_search_summary.to_csv(f'outputs/datasets/models/RULRegressor/GS_summary.csv',index=False)
grid_search_pipelines.to_csv(f'outputs/datasets/models/RULRegressor/GS_pipelines.csv',index=False)
linreg.to_csv(f'outputs/datasets/models/RULRegressor/RUL_linreg.csv',index=False)
ABreg.to_csv(f'outputs/datasets/models/RULRegressor/RUL_ABreg.csv',index=False)

df_total.to_csv(f'outputs/datasets/models/df_total.csv',index=False)
df_total_model.to_csv(f'outputs/datasets/models/df_total_model.csv',index=False)
df_train.to_csv(f'outputs/datasets/models/df_train.csv',index=False)
df_train_even_dist.to_csv(f'outputs/datasets/models/df_train_even_dist.csv',index=False)
df_test.to_csv(f'outputs/datasets/models/df_test.csv',index=False)